In [3]:
# Function to identify clips in reads

import pysam

def identify_clip(read):
    cigar = read.cigartuples
    seq = read.query_sequence

    clipped_seq = None
    if cigar[0][0] == 4:
        clipped_seq = seq[:cigar[0][1]]
    elif cigar[-1][0] == 4:
        clipped_seq = seq[-cigar[-1][1]:]

    return clipped_seq

In [4]:
# Function to get clipped quality scores

def get_clipped_qual(read, clipped_seq):
    qual_scores = read.query_qualities
    if read.cigartuples[0][0] == 4:
        clipped_qual = qual_scores[:read.cigartuples[0][1]]
    elif read.cigartuples[-1][0] == 4:
        clipped_qual = qual_scores[-read.cigartuples[-1][1]:]
    else:
        return None
    return "".join([chr(q + 33) for q in clipped_qual])

In [10]:
import pandas as pd
import pysam

CSV_PATH = "../results/filtered_NUMT_candidates.csv"
BAM_PATH = "../results/sample_alignment_sorted.bam"
CLIPPED_FASTQ = "../results/clipped_reads.fq"

df_clusters = pd.read_csv(CSV_PATH)
bam = pysam.AlignmentFile(BAM_PATH, "rb")

BIN_SIZE = 50
BUFFER = 1000
MIN_CLIP_LEN = 10

clipped_reads = []

for index, cluster in df_clusters.iterrows():
    chrom = cluster["ref_name"]
    start = max(0, cluster["bin_start"] - BUFFER)
    end = cluster["bin_start"] + BIN_SIZE + BUFFER

    for read in bam.fetch(chrom, start, end):
        if read.cigarstring is None:
            continue
        if "S" not in read.cigarstring:
            continue

        clipped_seq = identify_clip(read)
        if clipped_seq is None or len(clipped_seq) < MIN_CLIP_LEN:
            continue

        qual_str = get_clipped_qual(read, clipped_seq)
        if qual_str is None or len(qual_str) < MIN_CLIP_LEN:
            continue

        clipped_reads.append((read.query_name, clipped_seq, qual_str))

# Write filtered clipped reads to FASTQ
with open(CLIPPED_FASTQ, "w") as fq_out:
    for qname, seq, qual in clipped_reads:
        fq_out.write(f"@{qname}\n{seq}\n+\n{qual}\n")

print(f"Wrote {len(clipped_reads)} clipped reads to {CLIPPED_FASTQ}")

Wrote 1342 clipped reads to ../results/clipped_reads.fq


In [12]:
import subprocess

BWA_PATH = "bwa"
MTDNA_REF = "../data/human_mtDNA.fa"

SAM_FILE = "../results/clipped_vs_mtDNA.sam"
BAM_FILE = "../results/clipped_vs_mtDNA.bam"
SORTED_BAM = "../results/clipped_vs_mtDNA_sorted.bam"

subprocess.run([BWA_PATH, "mem", MTDNA_REF, CLIPPED_FASTQ, "-o", SAM_FILE], check=True)
subprocess.run(["samtools", "view", "-bS", SAM_FILE, "-o", BAM_FILE], check=True)
subprocess.run(["samtools", "sort", BAM_FILE, "-o", SORTED_BAM], check=True)
subprocess.run(["samtools", "index", SORTED_BAM], check=True)

print(f"Sorted BAM and index ready: {SORTED_BAM}")

Sorted BAM and index ready: ../results/clipped_vs_mtDNA_sorted.bam


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1342 sequences (38896 bp)...
[M::mem_process_seqs] Processed 1342 reads in 0.013 CPU sec, 0.017 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -o ../results/clipped_vs_mtDNA.sam ../data/human_mtDNA.fa ../results/clipped_reads.fq
[main] Real time: 0.041 sec; CPU: 0.027 sec
